In [1]:
import os
import numpy as np
import music21 as m21
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy import stats
from scipy import spatial
import time
from collections import Counter

np.random.seed(777)
# Define dataset paths
MXML_PATH="./dataset/MTC-ANN-2.0.1/musicxml/"
METADATA_PATH="./dataset/MTC-ANN-2.0.1/metadata/" 

DIV_CONST = 64

## Graphs

In [2]:
def autolabel(rects, ax):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(round(height,3)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

In [3]:
def plot_conf_noises(metric_values_, differences_percentages_, diff_x_noises_):
    fig, axs = plt.subplots(7, 1, figsize=(12, 6 * 7))
    # print(differences_percentages_)
    
    # Graphs per noise type
    for noise_i in range(len_noise_types):
        axs_to_plot = axs[noise_i]
        rects = axs_to_plot.bar(list(map(str, np.linspace(10, 100, 10))), differences_percentages_[noise_i])
        autolabel(rects, axs_to_plot)
        axs_to_plot.set_title(label="Noise #" + str(noise_i + 1))
    
    # Graph mean per percentage across noises
    grouped_percentage_values = []
    for percentage in map(int, np.linspace(10, 100, 10)):
        grouped_percentage_values.append([])
        
    for noise_i in range(len_noise_types):
        perc_i = 0
        for percentage in map(int, np.linspace(10, 100, 10)):
            grouped_percentage_values[perc_i].append(differences_percentages_[noise_i][perc_i])
            perc_i += 1
    perc_i = 0
    for percentage in map(int, np.linspace(10, 100, 10)):
        grouped_percentage_values[perc_i] = np.mean(grouped_percentage_values[perc_i])
        perc_i += 1
        
    axs_to_plot = axs[5]
    rects = axs_to_plot.bar(list(map(str, np.linspace(10, 100, 10))), grouped_percentage_values, color="orange")
    autolabel(rects, axs_to_plot)
    axs_to_plot.set_title(label="Mean Differences across all Noises")
    
    # Graph with mean for each noise accross percentages
    axs_to_plot = axs[6]
    rects = axs_to_plot.bar(list(range(1,6)), list(map(np.mean, differences_percentages_)), color="red")
    autolabel(rects, axs_to_plot)
    axs_to_plot.set_title(label="Mean Noise Difference")
        

In [4]:
def get_mean_diff_vectors(differences_percentages_):
    # Get Mean per percentage across noises
    grouped_percentage_values = []
    for percentage in map(int, np.linspace(10, 100, 10)):
        grouped_percentage_values.append([])
        
    for noise_i in range(len_noise_types):
        perc_i = 0
        for percentage in map(int, np.linspace(10, 100, 10)):
            grouped_percentage_values[perc_i].append(differences_percentages_[noise_i][perc_i])
            perc_i += 1
    perc_i = 0
    for percentage in map(int, np.linspace(10, 100, 10)):
        grouped_percentage_values[perc_i] = np.mean(grouped_percentage_values[perc_i])
        perc_i += 1
        
    return grouped_percentage_values, list(map(np.mean, differences_percentages_))

In [5]:
def plot_conf_best_noises(raw_calculations):
    fig, axs = plt.subplots(2, 1, figsize=(12, 6 * 2))
    differences_percentages_ = raw_calculations[1]
    # Graph mean per percentage across noises
    grouped_percentage_values = []
    for percentage in map(int, np.linspace(10, 100, 10)):
        grouped_percentage_values.append([])
        
    for noise_i in range(len_noise_types):
        perc_i = 0
        for percentage in map(int, np.linspace(10, 100, 10)):
            grouped_percentage_values[perc_i].append(differences_percentages_[noise_i][perc_i])
            perc_i += 1
    perc_i = 0
    for percentage in map(int, np.linspace(10, 100, 10)):
        grouped_percentage_values[perc_i] = np.mean(grouped_percentage_values[perc_i])
        perc_i += 1
        
    axs_to_plot = axs[0]
    rects = axs_to_plot.bar(list(map(str, np.linspace(10, 100, 10))), grouped_percentage_values, color="orange")
    autolabel(rects, axs_to_plot)
    axs_to_plot.set_title(label="Mean Differences across all Noises")
    
    # Graph with mean for each noisea accross percentages
    axs_to_plot = axs[1]
    rects = axs_to_plot.bar(list(range(1,6)), list(map(np.mean, differences_percentages_)), color="red")
    autolabel(rects, axs_to_plot)
    axs_to_plot.set_title(label="Mean Noise Difference")

## Dataset

In [6]:
# Read database of phrase similarity
phrase_similarity_filename = "MTC-ANN-phrase-similarity.csv"
phrase_similarity_df = pd.read_csv(os.path.join(METADATA_PATH, phrase_similarity_filename), header=None)

In [7]:
# Read table of tune family
tune_family_filename = "MTC-ANN-tune-family-labels.csv"
tune_family_df = pd.read_csv(os.path.join(METADATA_PATH, tune_family_filename), header=None)

In [8]:
# Traverse musicxml files and tune family
song_id_x_family = {}
family_x_songs = {}
for root, directories, files in os.walk(MXML_PATH):
    for file in files:
        song_id = file.split(".")[0]
        if (song_id not in song_id_x_family):
            family_name = tune_family_df[tune_family_df[0] == song_id].iloc[0][1]
            song_id_x_family[song_id] = (file, family_name)
            if (family_name not in family_x_songs):
                family_x_songs[family_name] = []
            family_x_songs[family_name].append(song_id)

In [9]:
# Remove the incomplete anotated tunes from the dataframe
reduced_tune_family_df = tune_family_df[tune_family_df[0].isin(list(song_id_x_family.keys()))]

## Functions

In [10]:
# Function to retrieve a list of midi pitch events and its timestamp
def getMelodyDeltaTimes(eventsintrack):
    
    # Initialize array
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Traverse the events
    for ev in eventsintrack:
        
        # If a note starts
        if (ev.isNoteOn()):
            
            # Get the pitch name and save it with the cumulative sum, midi pitch and name
            pitch_in_time = m21.pitch.Pitch(ev.pitch)
            DeltaTimes.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish))
            
            # Restart the delta time
            prev_deltatime = 0
        
        # Else if there is a delta time
        elif(str(ev.type) == "DeltaTime"):
            
            # We sum the time
            cum_sum += ev.time
            
            # We sum it to the current delta time
            prev_deltatime += ev.time
    
    # Return the array
    return DeltaTimes

In [11]:
# Function to retrieve a list of segments of midi pitch events and its timestamp from a stream
def getMelodyDeltaTimeSegmentsFromMelodyStream(melodystream):
    
    # Initialize array of arrays
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Current Segment
    curr_segment = []
    
    # Traverse the Notes in the stream
    for n in melodystream.parts[0].recurse().getElementsByClass('Note'):
        
        # We found a stop lyric
        if (n.lyric != None and n.lyric[0] == '!'):
            
            # We save the segment
            DeltaTimes.append(curr_segment)
            
            # We initialize the new segment
            curr_segment = []
            cum_sum = 0
            prev_deltatime = 0
        
        # We save the pitch
        pitch_in_time = m21.pitch.Pitch(n.pitch)
        curr_segment.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish, pitch_in_time))
        
        # We update the delta times
        cum_sum += m21.midi.translate.durationToMidiTicks(n.duration)
        prev_deltatime = m21.midi.translate.durationToMidiTicks(n.duration)
    
    # We save the last segment
    DeltaTimes.append(curr_segment)
    
    # We remove the initial empty list
    DeltaTimes.pop(0)
    
    # Return the array
    return DeltaTimes

In [12]:
def getSongKey(song):
    key = song.analyze("key")
    return key

In [13]:
def getSongKeyFromMelody_W_Times(melody_w_times_in_k):
    sc_test = m21.stream.Score()
    p0_test = m21.stream.Part()
    p0_test.id = 'part0'
    for pitch_i in melody_w_times_in_k:
        n_i = m21.note.Note(pitch_i[4])
        p0_test.append(n_i)
    sc_test.insert(0, p0_test)
    return getSongKey(sc_test)

In [14]:
def get_expert_score_from_tags(tag_i, tag_j):
    
    # If distinct letter no similarity
    if (tag_i[0] != tag_j[0]):
        return 1.0
    
    # If same letter and same number full similarity
    if (tag_i[1] == tag_j[1]):
        return 0.0
    
    # Otherwise "some" similarity
    return 0.5

In [15]:
def get_total_mean_dif_with_experts_by_song_ids(total_mean_differences_w_experts):
    total_scores_by_expert = [[], [], []]
    for song_id in total_mean_differences_w_experts:
        total_scores_by_expert[0].append(total_mean_differences_w_experts[song_id][0])
        total_scores_by_expert[1].append(total_mean_differences_w_experts[song_id][1])
        total_scores_by_expert[2].append(total_mean_differences_w_experts[song_id][2])
    total_mean_dif_score_1 = np.mean(total_scores_by_expert[0])
    total_mean_dif_score_2 = np.mean(total_scores_by_expert[1])
    total_mean_dif_score_3 = np.mean(total_scores_by_expert[2])
    return (total_mean_dif_score_1, total_mean_dif_score_2, total_mean_dif_score_3)

### SIAM

In [16]:
def get_SIAM_from_melody_w_times(melody_w_times_query, melody_w_times_test):
    
    if (len(melody_w_times_query) == 0 or len(melody_w_times_test) == 0):
        return 0
    
    start_onset = melody_w_times_query[0][0]
    query_points = [(s[0] - start_onset, s[2]) for s in melody_w_times_query]
    
    translation_vectors = []
        
    test_points = np.array([(s[0], s[2]) for s in melody_w_times_test])
    
    for p in query_points:
        vectors = (test_points - p)
        translation_vectors.extend([tuple(v) for v in vectors])
    
    grouped_vectors = dict(Counter(translation_vectors))
    similarity = max([grouped_vectors[k] for k in grouped_vectors])     
    
    return similarity / float(len(query_points))

In [17]:
%%time

# weights
(W1, W2, W3, W4) = (0.0,0.0,1.0,0.0)

# Store Dict
scores_metric_and_experts = {}

# Family list
family_names = reduced_tune_family_df[1].unique()
cnt_mels = 1
cnt_fams = 1
TOT_FAMS = len(family_names)

for family_name in family_names:
    # print("FAM: {0} {1}/{2}".format(family_name, cnt_fams, TOT_FAMS))
    current_family_melodies_df = reduced_tune_family_df[reduced_tune_family_df[1] == family_name]
    
    TOT_LOCAL_COMPS = len(current_family_melodies_df[0]) ** 2
    cnt_local_comps = 1
    for melody_id_i in current_family_melodies_df[0]:
        for melody_id_j in current_family_melodies_df[0]:
            
            current_test_name = melody_id_i + ":" + melody_id_j
            
            print("TESTING: {0} {1}/{2} FAM: {3}/{4}".format(current_test_name, cnt_local_comps, TOT_LOCAL_COMPS, cnt_fams, TOT_FAMS))
            
            scores_metric_and_experts[current_test_name] = []
            
            # We get the stream from file
            melody_stream_i = m21.converter.parseFile(os.path.join(MXML_PATH, melody_id_i + ".musicxml"))

            # We get the melody delta segments
            melody_deltas_i = getMelodyDeltaTimeSegmentsFromMelodyStream(melody_stream_i)

            # We get the current tag by experts
            exp_tag_1_i = phrase_similarity_df[phrase_similarity_df[0] == melody_id_i][2]
            exp_tag_2_i = phrase_similarity_df[phrase_similarity_df[0] == melody_id_i][3]
            exp_tag_3_i = phrase_similarity_df[phrase_similarity_df[0] == melody_id_i][4]
            
            # Get Limits
            len_mdlts_i = len(melody_deltas_i)
            
            # J
            # We get the stream from file 
            melody_stream_j = m21.converter.parseFile(os.path.join(MXML_PATH, melody_id_j + ".musicxml"))

            # We get the melody delta segments
            melody_deltas_j = getMelodyDeltaTimeSegmentsFromMelodyStream(melody_stream_j)

            # We get the current tag by experts
            exp_tag_1_j = phrase_similarity_df[phrase_similarity_df[0] == melody_id_j][2]
            exp_tag_2_j = phrase_similarity_df[phrase_similarity_df[0] == melody_id_j][3]
            exp_tag_3_j = phrase_similarity_df[phrase_similarity_df[0] == melody_id_j][4]
            
            # Get Limits
            len_mdlts_j = len(melody_deltas_j)
            
            # We calculate the distance between each pair of segments
            for i in range(len_mdlts_i):
                for j in range(len_mdlts_j):

                    # We get the metric value for the current pair
                    calculated_mtrc = get_SIAM_from_melody_w_times(melody_deltas_i[i], melody_deltas_j[j])
                    # calculated_mtrc = get_metric_range_to_100(calculated_mtrc)

                    # We get the score by the experts on the current pair
                    expert_score_1 = get_expert_score_from_tags(exp_tag_1_i.iloc[i], exp_tag_1_j.iloc[j])
                    expert_score_2 = get_expert_score_from_tags(exp_tag_2_i.iloc[i], exp_tag_2_j.iloc[j])
                    expert_score_3 = get_expert_score_from_tags(exp_tag_3_i.iloc[i], exp_tag_3_j.iloc[j])
                    # print((expert_score_1, exp_tag_1.iloc[i], exp_tag_1.iloc[j]))

                    # We save the scores
                    scores_metric_and_experts[current_test_name].append((
                        calculated_mtrc,
                        expert_score_1,
                        expert_score_2,
                        expert_score_3
                    ))
                    cnt_mels += 1
            cnt_local_comps += 1
    cnt_fams += 1
    print("Comparisons made: {0}".format(cnt_mels))

TESTING: NLB072587_01:NLB072587_01 1/256 FAM: 1/26
TESTING: NLB072587_01:NLB072587_02 2/256 FAM: 1/26
TESTING: NLB072587_01:NLB072774_02 3/256 FAM: 1/26
TESTING: NLB072587_01:NLB073046_01 4/256 FAM: 1/26
TESTING: NLB072587_01:NLB073588_01 5/256 FAM: 1/26
TESTING: NLB072587_01:NLB073672_01 6/256 FAM: 1/26
TESTING: NLB072587_01:NLB073681_01 7/256 FAM: 1/26
TESTING: NLB072587_01:NLB073743_01 8/256 FAM: 1/26
TESTING: NLB072587_01:NLB073822_01 9/256 FAM: 1/26
TESTING: NLB072587_01:NLB074004_01 10/256 FAM: 1/26
TESTING: NLB072587_01:NLB074048_02 11/256 FAM: 1/26
TESTING: NLB072587_01:NLB074227_01 12/256 FAM: 1/26
TESTING: NLB072587_01:NLB075551_01 13/256 FAM: 1/26
TESTING: NLB072587_01:NLB076625_01 14/256 FAM: 1/26
TESTING: NLB072587_01:NLB076632_01 15/256 FAM: 1/26
TESTING: NLB072587_01:NLB144072_01 16/256 FAM: 1/26
TESTING: NLB072587_02:NLB072587_01 17/256 FAM: 1/26
TESTING: NLB072587_02:NLB072587_02 18/256 FAM: 1/26
TESTING: NLB072587_02:NLB072774_02 19/256 FAM: 1/26
TESTING: NLB072587_02

TESTING: NLB074048_02:NLB073743_01 168/256 FAM: 1/26
TESTING: NLB074048_02:NLB073822_01 169/256 FAM: 1/26
TESTING: NLB074048_02:NLB074004_01 170/256 FAM: 1/26
TESTING: NLB074048_02:NLB074048_02 171/256 FAM: 1/26
TESTING: NLB074048_02:NLB074227_01 172/256 FAM: 1/26
TESTING: NLB074048_02:NLB075551_01 173/256 FAM: 1/26
TESTING: NLB074048_02:NLB076625_01 174/256 FAM: 1/26
TESTING: NLB074048_02:NLB076632_01 175/256 FAM: 1/26
TESTING: NLB074048_02:NLB144072_01 176/256 FAM: 1/26
TESTING: NLB074227_01:NLB072587_01 177/256 FAM: 1/26
TESTING: NLB074227_01:NLB072587_02 178/256 FAM: 1/26
TESTING: NLB074227_01:NLB072774_02 179/256 FAM: 1/26
TESTING: NLB074227_01:NLB073046_01 180/256 FAM: 1/26
TESTING: NLB074227_01:NLB073588_01 181/256 FAM: 1/26
TESTING: NLB074227_01:NLB073672_01 182/256 FAM: 1/26
TESTING: NLB074227_01:NLB073681_01 183/256 FAM: 1/26
TESTING: NLB074227_01:NLB073743_01 184/256 FAM: 1/26
TESTING: NLB074227_01:NLB073822_01 185/256 FAM: 1/26
TESTING: NLB074227_01:NLB074004_01 186/256 FAM

TESTING: NLB073929_01:NLB072912_01 75/144 FAM: 2/26
TESTING: NLB073929_01:NLB072920_01 76/144 FAM: 2/26
TESTING: NLB073929_01:NLB072946_01 77/144 FAM: 2/26
TESTING: NLB073929_01:NLB073426_01 78/144 FAM: 2/26
TESTING: NLB073929_01:NLB073929_01 79/144 FAM: 2/26
TESTING: NLB073929_01:NLB074028_01 80/144 FAM: 2/26
TESTING: NLB073929_01:NLB111656_01 81/144 FAM: 2/26
TESTING: NLB073929_01:NLB112210_01 82/144 FAM: 2/26
TESTING: NLB073929_01:NLB141251_01 83/144 FAM: 2/26
TESTING: NLB073929_01:NLB144042_01 84/144 FAM: 2/26
TESTING: NLB074028_01:NLB070801_01 85/144 FAM: 2/26
TESTING: NLB074028_01:NLB072154_01 86/144 FAM: 2/26
TESTING: NLB074028_01:NLB072912_01 87/144 FAM: 2/26
TESTING: NLB074028_01:NLB072920_01 88/144 FAM: 2/26
TESTING: NLB074028_01:NLB072946_01 89/144 FAM: 2/26
TESTING: NLB074028_01:NLB073426_01 90/144 FAM: 2/26
TESTING: NLB074028_01:NLB073929_01 91/144 FAM: 2/26
TESTING: NLB074028_01:NLB074028_01 92/144 FAM: 2/26
TESTING: NLB074028_01:NLB111656_01 93/144 FAM: 2/26
TESTING: NLB

TESTING: NLB073287_01:NLB073990_01 92/256 FAM: 3/26
TESTING: NLB073287_01:NLB074328_01 93/256 FAM: 3/26
TESTING: NLB073287_01:NLB074427_01 94/256 FAM: 3/26
TESTING: NLB073287_01:NLB074552_01 95/256 FAM: 3/26
TESTING: NLB073287_01:NLB076211_01 96/256 FAM: 3/26
TESTING: NLB073287_02:NLB070493_01 97/256 FAM: 3/26
TESTING: NLB073287_02:NLB072003_01 98/256 FAM: 3/26
TESTING: NLB073287_02:NLB072015_01 99/256 FAM: 3/26
TESTING: NLB073287_02:NLB072627_01 100/256 FAM: 3/26
TESTING: NLB073287_02:NLB072690_01 101/256 FAM: 3/26
TESTING: NLB073287_02:NLB073287_01 102/256 FAM: 3/26
TESTING: NLB073287_02:NLB073287_02 103/256 FAM: 3/26
TESTING: NLB073287_02:NLB073337_02 104/256 FAM: 3/26
TESTING: NLB073287_02:NLB073483_01 105/256 FAM: 3/26
TESTING: NLB073287_02:NLB073628_01 106/256 FAM: 3/26
TESTING: NLB073287_02:NLB073639_01 107/256 FAM: 3/26
TESTING: NLB073287_02:NLB073990_01 108/256 FAM: 3/26
TESTING: NLB073287_02:NLB074328_01 109/256 FAM: 3/26
TESTING: NLB073287_02:NLB074427_01 110/256 FAM: 3/26
T

Comparisons made: 10802
TESTING: NLB015569_01:NLB015569_01 1/81 FAM: 4/26
TESTING: NLB015569_01:NLB071441_01 2/81 FAM: 4/26
TESTING: NLB015569_01:NLB071666_01 3/81 FAM: 4/26
TESTING: NLB015569_01:NLB072299_01 4/81 FAM: 4/26
TESTING: NLB015569_01:NLB072306_01 5/81 FAM: 4/26
TESTING: NLB015569_01:NLB072311_01 6/81 FAM: 4/26
TESTING: NLB015569_01:NLB072886_01 7/81 FAM: 4/26
TESTING: NLB015569_01:NLB073150_01 8/81 FAM: 4/26
TESTING: NLB015569_01:NLB138219_01 9/81 FAM: 4/26
TESTING: NLB071441_01:NLB015569_01 10/81 FAM: 4/26
TESTING: NLB071441_01:NLB071441_01 11/81 FAM: 4/26
TESTING: NLB071441_01:NLB071666_01 12/81 FAM: 4/26
TESTING: NLB071441_01:NLB072299_01 13/81 FAM: 4/26
TESTING: NLB071441_01:NLB072306_01 14/81 FAM: 4/26
TESTING: NLB071441_01:NLB072311_01 15/81 FAM: 4/26
TESTING: NLB071441_01:NLB072886_01 16/81 FAM: 4/26
TESTING: NLB071441_01:NLB073150_01 17/81 FAM: 4/26
TESTING: NLB071441_01:NLB138219_01 18/81 FAM: 4/26
TESTING: NLB071666_01:NLB015569_01 19/81 FAM: 4/26
TESTING: NLB0716

...

TESTING: NLB073393_01:NLB074468_01 169/289 FAM: 23/26
TESTING: NLB073393_01:NLB074954_01 170/289 FAM: 23/26
TESTING: NLB073888_01:NLB072103_01 171/289 FAM: 23/26
TESTING: NLB073888_01:NLB072283_01 172/289 FAM: 23/26
TESTING: NLB073888_01:NLB072284_01 173/289 FAM: 23/26
TESTING: NLB073888_01:NLB072285_01 174/289 FAM: 23/26
TESTING: NLB073888_01:NLB072286_01 175/289 FAM: 23/26
TESTING: NLB073888_01:NLB072287_01 176/289 FAM: 23/26
TESTING: NLB073888_01:NLB072288_01 177/289 FAM: 23/26
TESTING: NLB073888_01:NLB072289_01 178/289 FAM: 23/26
TESTING: NLB073888_01:NLB073311_01 179/289 FAM: 23/26
TESTING: NLB073888_01:NLB073393_01 180/289 FAM: 23/26
TESTING: NLB073888_01:NLB073888_01 181/289 FAM: 23/26
TESTING: NLB073888_01:NLB073992_01 182/289 FAM: 23/26
TESTING: NLB073888_01:NLB074157_01 183/289 FAM: 23/26
TESTING: NLB073888_01:NLB074161_01 184/289 FAM: 23/26
TESTING: NLB073888_01:NLB074234_01 185/289 FAM: 23/26
TESTING: NLB073888_01:NLB074468_01 186/289 FAM: 23/26
TESTING: NLB073888_01:NLB074

TESTING: NLB072499_01:NLB162526_01 33/121 FAM: 24/26
TESTING: NLB073210_01:NLB070732_01 34/121 FAM: 24/26
TESTING: NLB073210_01:NLB071369_01 35/121 FAM: 24/26
TESTING: NLB073210_01:NLB072499_01 36/121 FAM: 24/26
TESTING: NLB073210_01:NLB073210_01 37/121 FAM: 24/26
TESTING: NLB073210_01:NLB075158_01 38/121 FAM: 24/26
TESTING: NLB073210_01:NLB076130_07 39/121 FAM: 24/26
TESTING: NLB073210_01:NLB112123_01 40/121 FAM: 24/26
TESTING: NLB073210_01:NLB134389_01 41/121 FAM: 24/26
TESTING: NLB073210_01:NLB144100_01 42/121 FAM: 24/26
TESTING: NLB073210_01:NLB162519_01 43/121 FAM: 24/26
TESTING: NLB073210_01:NLB162526_01 44/121 FAM: 24/26
TESTING: NLB075158_01:NLB070732_01 45/121 FAM: 24/26
TESTING: NLB075158_01:NLB071369_01 46/121 FAM: 24/26
TESTING: NLB075158_01:NLB072499_01 47/121 FAM: 24/26
TESTING: NLB075158_01:NLB073210_01 48/121 FAM: 24/26
TESTING: NLB075158_01:NLB075158_01 49/121 FAM: 24/26
TESTING: NLB075158_01:NLB076130_07 50/121 FAM: 24/26
TESTING: NLB075158_01:NLB112123_01 51/121 FAM:

TESTING: NLB072691_01:NLB070492_01 71/196 FAM: 25/26
TESTING: NLB072691_01:NLB071944_01 72/196 FAM: 25/26
TESTING: NLB072691_01:NLB072565_01 73/196 FAM: 25/26
TESTING: NLB072691_01:NLB072665_01 74/196 FAM: 25/26
TESTING: NLB072691_01:NLB072688_01 75/196 FAM: 25/26
TESTING: NLB072691_01:NLB072691_01 76/196 FAM: 25/26
TESTING: NLB072691_01:NLB072714_01 77/196 FAM: 25/26
TESTING: NLB072691_01:NLB073286_01 78/196 FAM: 25/26
TESTING: NLB072691_01:NLB073771_02 79/196 FAM: 25/26
TESTING: NLB072691_01:NLB073991_02 80/196 FAM: 25/26
TESTING: NLB072691_01:NLB073997_01 81/196 FAM: 25/26
TESTING: NLB072691_01:NLB074962_01 82/196 FAM: 25/26
TESTING: NLB072691_01:NLB075064_01 83/196 FAM: 25/26
TESTING: NLB072691_01:NLB075068_01 84/196 FAM: 25/26
TESTING: NLB072714_01:NLB070492_01 85/196 FAM: 25/26
TESTING: NLB072714_01:NLB071944_01 86/196 FAM: 25/26
TESTING: NLB072714_01:NLB072565_01 87/196 FAM: 25/26
TESTING: NLB072714_01:NLB072665_01 88/196 FAM: 25/26
TESTING: NLB072714_01:NLB072688_01 89/196 FAM:

TESTING: NLB070096_01:NLB134474_01 35/324 FAM: 26/26
TESTING: NLB070096_01:NLB141314_01 36/324 FAM: 26/26
TESTING: NLB070748_01:NLB070053_01 37/324 FAM: 26/26
TESTING: NLB070748_01:NLB070096_01 38/324 FAM: 26/26
TESTING: NLB070748_01:NLB070748_01 39/324 FAM: 26/26
TESTING: NLB070748_01:NLB071014_01 40/324 FAM: 26/26
TESTING: NLB070748_01:NLB071064_01 41/324 FAM: 26/26
TESTING: NLB070748_01:NLB071082_01 42/324 FAM: 26/26
TESTING: NLB070748_01:NLB073225_01 43/324 FAM: 26/26
TESTING: NLB070748_01:NLB073225_02 44/324 FAM: 26/26
TESTING: NLB070748_01:NLB073277_01 45/324 FAM: 26/26
TESTING: NLB070748_01:NLB073298_01 46/324 FAM: 26/26
TESTING: NLB070748_01:NLB073626_01 47/324 FAM: 26/26
TESTING: NLB070748_01:NLB073897_01 48/324 FAM: 26/26
TESTING: NLB070748_01:NLB074100_01 49/324 FAM: 26/26
TESTING: NLB070748_01:NLB074182_01 50/324 FAM: 26/26
TESTING: NLB070748_01:NLB075073_01 51/324 FAM: 26/26
TESTING: NLB070748_01:NLB111760_01 52/324 FAM: 26/26
TESTING: NLB070748_01:NLB134474_01 53/324 FAM:

TESTING: NLB073626_01:NLB073897_01 192/324 FAM: 26/26
TESTING: NLB073626_01:NLB074100_01 193/324 FAM: 26/26
TESTING: NLB073626_01:NLB074182_01 194/324 FAM: 26/26
TESTING: NLB073626_01:NLB075073_01 195/324 FAM: 26/26
TESTING: NLB073626_01:NLB111760_01 196/324 FAM: 26/26
TESTING: NLB073626_01:NLB134474_01 197/324 FAM: 26/26
TESTING: NLB073626_01:NLB141314_01 198/324 FAM: 26/26
TESTING: NLB073897_01:NLB070053_01 199/324 FAM: 26/26
TESTING: NLB073897_01:NLB070096_01 200/324 FAM: 26/26
TESTING: NLB073897_01:NLB070748_01 201/324 FAM: 26/26
TESTING: NLB073897_01:NLB071014_01 202/324 FAM: 26/26
TESTING: NLB073897_01:NLB071064_01 203/324 FAM: 26/26
TESTING: NLB073897_01:NLB071082_01 204/324 FAM: 26/26
TESTING: NLB073897_01:NLB073225_01 205/324 FAM: 26/26
TESTING: NLB073897_01:NLB073225_02 206/324 FAM: 26/26
TESTING: NLB073897_01:NLB073277_01 207/324 FAM: 26/26
TESTING: NLB073897_01:NLB073298_01 208/324 FAM: 26/26
TESTING: NLB073897_01:NLB073626_01 209/324 FAM: 26/26
TESTING: NLB073897_01:NLB073

In [18]:
ts = time.time()
with open('./ROC_SIAM_{0}.json'.format(str(ts)), 'w') as outfile:
    json.dump(scores_metric_and_experts, outfile)